# Using transformers for SeriesBasedCesnetDataset

### Import

In [1]:
import numpy as np
import logging

from cesnet_tszoo.utils.enums import AgreggationType, SourceType, TransformerType, DatasetType
from cesnet_tszoo.datasets import CESNET_TimeSeries24
from cesnet_tszoo.configs import SeriesBasedConfig # Series based dataset MUST use SeriesBasedConfig

from cesnet_tszoo.utils.transformer import Transformer # For creating custom Transformer

### Setting logger

In [2]:
logging.basicConfig(
    level=logging.INFO,
    format="[%(asctime)s][%(name)s][%(levelname)s] - %(message)s")

### Preparing dataset

In [3]:
series_based_dataset = CESNET_TimeSeries24.get_dataset(data_root="/some_directory/", source_type=SourceType.IP_ADDRESSES_SAMPLE, aggregation=AgreggationType.AGG_10_MINUTES, dataset_type=DatasetType.SERIES_BASED, display_details=True)

[2025-08-26 20:07:43,874][wrapper_dataset][INFO] - Dataset is series-based. Use cesnet_tszoo.configs.SeriesBasedConfig



Dataset details:

    AgreggationType.AGG_10_MINUTES
        Time indices: range(0, 40297)
        Datetime: (datetime.datetime(2023, 10, 9, 0, 3, 49, tzinfo=datetime.timezone.utc), datetime.datetime(2024, 7, 14, 21, 50, 52, tzinfo=datetime.timezone.utc))

    SourceType.IP_ADDRESSES_SAMPLE
        Time series indices: [ 11  20 101 103 118 ... 2003134 2008461 2011839 2022235 2044888], Length=1000; use 'get_available_ts_indices' for full list
        Features with default values: {'n_flows': 0, 'n_packets': 0, 'n_bytes': 0, 'n_dest_ip': 0, 'n_dest_asn': 0, 'n_dest_ports': 0, 'tcp_udp_ratio_packets': 0.5, 'tcp_udp_ratio_bytes': 0.5, 'dir_ratio_packets': 0.5, 'dir_ratio_bytes': 0.5, 'avg_duration': 0, 'avg_ttl': 0}
        
        Additional data: ['ids_relationship', 'weekends_and_holidays']
        


### Transformers

- Transformers are implemented as class.
    - You can create your own or use built-in one.
- Transformer is applied after `default_values` and fillers took care of missing values.
- One transformer is used for all time series.
- Transformer must implement `transform`.
- Transformer must implement `partial_fit` (unless transformer is already fitted and `partial_fit_initialized_transformers` is False).
- To use transformer, train set must be implemented (unless transformer is already fitted and `partial_fit_initialized_transformers` is False).
- You can change used transformer later with `update_dataset_config_and_initialize` or `apply_transformer`.

#### Built-in

In [4]:
# Options

## Supported
TransformerType.STANDARD_SCALER
TransformerType.L2_NORMALIZER
TransformerType.LOG_TRANSFORMER
TransformerType.MAX_ABS_SCALER
TransformerType.MIN_MAX_SCALER

<TransformerType.MIN_MAX_SCALER: 'min_max_scaler'>

In [5]:
config = SeriesBasedConfig(time_period=0.5, train_ts=500, features_to_take=["n_flows", "n_packets"],
                           transform_with=TransformerType.MIN_MAX_SCALER, nan_threshold=0.5, random_state=1500)
series_based_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-26 20:07:43,885][series_config][INFO] - Quick validation succeeded.
[2025-08-26 20:07:43,917][series_config][INFO] - Finalization and validation completed successfully.
[2025-08-26 20:07:43,921][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time period.
100%|██████████| 500/500 [00:00<00:00, 921.43it/s]
[2025-08-26 20:07:44,475][cesnet_dataset][INFO] - Dataset initialization complete. Configuration updated.
[2025-08-26 20:07:44,476][cesnet_dataset][INFO] - Config initialized successfully.



Config Details:
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_10_MINUTES
    Source: SourceType.IP_ADDRESSES_SAMPLE

    Time series
        Train time series IDS: [182151  10158  65072  10196 338309 ... 175742 659213  11188  73422 483796], Length=60
        Val time series IDS: None
        Test time series IDS None
        All time series IDS [182151  10158  65072  10196 338309 ... 175742 659213  11188  73422 483796], Length=60
    Time periods
        Time period: range(0, 20149)
    Features
        Taken features: ['n_flows', 'n_packets']
        Default values: [0. 0.]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.ID_TIME
    Fillers         
        Filler type: None
    Transformers
        Transformer type: min_max_scaler
        Are transformers premade: False
        Are premade transformers partial_fitted: False
    Anomaly handler
        Anomaly handler type (train set): None   
   

In [6]:
series_based_dataset.get_train_df(workers=0).head(10)

,id_ip,id_time,n_flows,n_packets
0,182151.0,0.0,0.000079,2.240487e-07
1,182151.0,1.0,0.000045,1.280278e-07
2,182151.0,2.0,0.000045,1.280278e-07
3,182151.0,3.0,0.000056,1.600348e-07
4,182151.0,4.0,0.000090,2.560557e-07
5,182151.0,5.0,0.000034,9.602087e-08
6,182151.0,6.0,0.000068,1.920417e-07
7,182151.0,7.0,0.000045,1.280278e-07
8,182151.0,8.0,0.000102,3.840835e-07
9,182151.0,9.0,0.000000,0.000000e+00


In [7]:
series_based_dataset.get_transformers()

Or later with:

In [8]:
series_based_dataset.update_dataset_config_and_initialize(transform_with=TransformerType.MIN_MAX_SCALER, partial_fit_initialized_transformers="config", workers=0)
# Or
series_based_dataset.apply_transformer(transform_with=TransformerType.MIN_MAX_SCALER, partial_fit_initialized_transformers="config", workers=0)

[2025-08-26 20:07:44,692][cesnet_dataset][INFO] - Re-initialization is required.
[2025-08-26 20:07:44,722][series_config][INFO] - Finalization and validation completed successfully.
[2025-08-26 20:07:44,725][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time period.
100%|██████████| 60/60 [00:00<00:00, 368.87it/s]
[2025-08-26 20:07:44,890][cesnet_dataset][INFO] - Dataset initialization complete. Configuration updated.
[2025-08-26 20:07:44,890][cesnet_dataset][INFO] - Config initialized successfully.
[2025-08-26 20:07:44,891][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-08-26 20:07:44,891][cesnet_dataset][INFO] - Re-initialization is required.
[2025-08-26 20:07:44,922][series_config][INFO] - Finalization and validation completed successfully.
[2025-08-26 20:07:44,926][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time period.
100%|██████████| 60/60 [00:00<00:00, 367.13it/s]
[2025-08-26 20:07:45,090][c

#### Custom

You can create your own custom transformer. It is recommended to derive from Transformer base class.

In [9]:
class CustomTransformer(Transformer):
    def __init__(self):
        super().__init__()
        
        self.max = None
        self.min = None
    
    def transform(self, data):
        return (data - self.min) / (self.max - self.min)
    
    def fit(self, data):
        self.partial_fit(data)
    
    def partial_fit(self, data):
        
        if self.max is None and self.min is None:
            self.max = np.max(data, axis=0)
            self.min = np.min(data, axis=0)
            return
        
        temp_max = np.max(data, axis=0)
        temp = np.vstack((self.max, temp_max)) 
        self.max = np.max(temp, axis=0)
        
        temp_min = np.min(data, axis=0)
        temp = np.vstack((self.min, temp_min)) 
        self.min = np.min(temp, axis=0)            

In [10]:
config = SeriesBasedConfig(time_period=0.5, train_ts=500, features_to_take=["n_flows", "n_packets"],
                           transform_with=CustomTransformer, nan_threshold=0.5, random_state=1500)
series_based_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-26 20:07:45,102][series_config][INFO] - Quick validation succeeded.
[2025-08-26 20:07:45,134][series_config][INFO] - Finalization and validation completed successfully.
[2025-08-26 20:07:45,138][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time period.
100%|██████████| 500/500 [00:00<00:00, 1051.73it/s]
[2025-08-26 20:07:45,616][cesnet_dataset][INFO] - Dataset initialization complete. Configuration updated.
[2025-08-26 20:07:45,616][cesnet_dataset][INFO] - Config initialized successfully.



Config Details:
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_10_MINUTES
    Source: SourceType.IP_ADDRESSES_SAMPLE

    Time series
        Train time series IDS: [182151  10158  65072  10196 338309 ... 175742 659213  11188  73422 483796], Length=60
        Val time series IDS: None
        Test time series IDS None
        All time series IDS [182151  10158  65072  10196 338309 ... 175742 659213  11188  73422 483796], Length=60
    Time periods
        Time period: range(0, 20149)
    Features
        Taken features: ['n_flows', 'n_packets']
        Default values: [0. 0.]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.ID_TIME
    Fillers         
        Filler type: None
    Transformers
        Transformer type: CustomTransformer (Custom)
        Are transformers premade: False
        Are premade transformers partial_fitted: False
    Anomaly handler
        Anomaly handler type (train set):

In [11]:
series_based_dataset.get_train_df(workers=0).head(10)

,id_ip,id_time,n_flows,n_packets
0,182151.0,0.0,0.000079,2.240487e-07
1,182151.0,1.0,0.000045,1.280278e-07
2,182151.0,2.0,0.000045,1.280278e-07
3,182151.0,3.0,0.000056,1.600348e-07
4,182151.0,4.0,0.000090,2.560557e-07
5,182151.0,5.0,0.000034,9.602087e-08
6,182151.0,6.0,0.000068,1.920417e-07
7,182151.0,7.0,0.000045,1.280278e-07
8,182151.0,8.0,0.000102,3.840835e-07
9,182151.0,9.0,0.000000,0.000000e+00


In [12]:
series_based_dataset.get_transformers()

Or later with:

In [13]:
series_based_dataset.update_dataset_config_and_initialize(transform_with=CustomTransformer, partial_fit_initialized_transformers="config", workers=0)
# Or
series_based_dataset.apply_transformer(transform_with=CustomTransformer, partial_fit_initialized_transformers="config", workers=0)

[2025-08-26 20:07:45,800][cesnet_dataset][INFO] - Re-initialization is required.
[2025-08-26 20:07:45,832][series_config][INFO] - Finalization and validation completed successfully.
[2025-08-26 20:07:45,835][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time period.
100%|██████████| 60/60 [00:00<00:00, 390.49it/s]
[2025-08-26 20:07:45,989][cesnet_dataset][INFO] - Dataset initialization complete. Configuration updated.
[2025-08-26 20:07:45,990][cesnet_dataset][INFO] - Config initialized successfully.
[2025-08-26 20:07:45,990][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-08-26 20:07:45,991][cesnet_dataset][INFO] - Re-initialization is required.
[2025-08-26 20:07:46,023][series_config][INFO] - Finalization and validation completed successfully.
[2025-08-26 20:07:46,026][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time period.
100%|██████████| 60/60 [00:00<00:00, 392.92it/s]
[2025-08-26 20:07:46,180][c

#### Using already fitted transformer

- When `partial_fit_initialized_transformer` is False (default value), transformer has no requirement for `partial_fit` nor for train set.

In [14]:
config = SeriesBasedConfig(time_period=0.5, train_ts=500, features_to_take=["n_flows", "n_packets"],
                           transform_with=CustomTransformer, nan_threshold=0.5, random_state=1500)
series_based_dataset.set_dataset_config_and_initialize(config, display_config_details=False, workers=0)

fitted_transformer = series_based_dataset.get_transformers()

[2025-08-26 20:07:46,186][series_config][INFO] - Quick validation succeeded.
[2025-08-26 20:07:46,270][series_config][INFO] - Finalization and validation completed successfully.
[2025-08-26 20:07:46,273][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time period.
100%|██████████| 500/500 [00:00<00:00, 927.39it/s] 
[2025-08-26 20:07:46,814][cesnet_dataset][INFO] - Dataset initialization complete. Configuration updated.
[2025-08-26 20:07:46,814][cesnet_dataset][INFO] - Config initialized successfully.


In [15]:
config = SeriesBasedConfig(time_period=0.5, train_ts=500, val_ts=500, features_to_take=["n_flows", "n_packets"],
                           transform_with=fitted_transformer, nan_threshold=0.5, random_state=999)
series_based_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-26 20:07:46,819][series_config][INFO] - Quick validation succeeded.
[2025-08-26 20:07:46,851][series_config][INFO] - Finalization and validation completed successfully.
[2025-08-26 20:07:46,854][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time period.
100%|██████████| 1000/1000 [00:00<00:00, 1157.82it/s]
[2025-08-26 20:07:47,720][cesnet_dataset][INFO] - Dataset initialization complete. Configuration updated.
[2025-08-26 20:07:47,720][cesnet_dataset][INFO] - Config initialized successfully.



Config Details:
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_10_MINUTES
    Source: SourceType.IP_ADDRESSES_SAMPLE

    Time series
        Train time series IDS: [  4380  35210 322201    190 307400 ...  29194 617662 677144 211973 612051], Length=65
        Val time series IDS: [121596 605036  67484   1852   6794 ... 564287 360850 363446 151737 811185], Length=70
        Test time series IDS None
        All time series IDS [  4380  35210 322201    190 307400 ... 564287 360850 363446 151737 811185], Length=135
    Time periods
        Time period: range(0, 20149)
    Features
        Taken features: ['n_flows', 'n_packets']
        Default values: [0. 0.]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.ID_TIME
    Fillers         
        Filler type: None
    Transformers
        Transformer type: CustomTransformer (Custom)
        Are transformers premade: True
        Are premade transformers p

In [16]:
series_based_dataset.get_val_df(workers=0).head(10)

,id_ip,id_time,n_flows,n_packets
0,121596.0,0.0,0.000237,0.000006
1,121596.0,1.0,0.000361,0.000009
2,121596.0,2.0,0.000361,0.000010
3,121596.0,3.0,0.000282,0.000008
4,121596.0,4.0,0.000316,0.000010
5,121596.0,5.0,0.000395,0.000009
6,121596.0,6.0,0.000361,0.000010
7,121596.0,7.0,0.000361,0.000008
8,121596.0,8.0,0.000350,0.000010
9,121596.0,9.0,0.000248,0.000005


Below you can see how transformer works even without train set.

In [17]:
config = SeriesBasedConfig(time_period=0.5, val_ts=500, features_to_take=["n_flows", "n_packets"],
                           transform_with=fitted_transformer, nan_threshold=0.5, random_state=999)
series_based_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-26 20:07:47,939][series_config][INFO] - Quick validation succeeded.
[2025-08-26 20:07:47,969][series_config][INFO] - Finalization and validation completed successfully.
[2025-08-26 20:07:47,973][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time period.
100%|██████████| 500/500 [00:00<00:00, 1138.34it/s]
[2025-08-26 20:07:48,415][cesnet_dataset][INFO] - Dataset initialization complete. Configuration updated.
[2025-08-26 20:07:48,415][cesnet_dataset][INFO] - Config initialized successfully.



Config Details:
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_10_MINUTES
    Source: SourceType.IP_ADDRESSES_SAMPLE

    Time series
        Train time series IDS: None
        Val time series IDS: [  4380  35210 322201    190 307400 ...  29194 617662 677144 211973 612051], Length=65
        Test time series IDS None
        All time series IDS [  4380  35210 322201    190 307400 ...  29194 617662 677144 211973 612051], Length=65
    Time periods
        Time period: range(0, 20149)
    Features
        Taken features: ['n_flows', 'n_packets']
        Default values: [0. 0.]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.ID_TIME
    Fillers         
        Filler type: None
    Transformers
        Transformer type: CustomTransformer (Custom)
        Are transformers premade: True
        Are premade transformers partial_fitted: False
    Anomaly handler
        Anomaly handler type (train set): 

In [18]:
series_based_dataset.get_val_df(workers=0).head(10)

,id_ip,id_time,n_flows,n_packets
0,4380.0,0.0,0.001253,0.000005
1,4380.0,1.0,0.001162,0.000004
2,4380.0,2.0,0.000756,0.000003
3,4380.0,3.0,0.000959,0.000004
4,4380.0,4.0,0.001219,0.000005
5,4380.0,5.0,0.001275,0.000005
6,4380.0,6.0,0.001456,0.000006
7,4380.0,7.0,0.001196,0.000004
8,4380.0,8.0,0.001433,0.000006
9,4380.0,9.0,0.001072,0.000004


##### Partial fitting on train set

Makes already fitted transformer to be fitted on new train set too. Must implement `partial_fit`.

In [19]:
config = SeriesBasedConfig(time_period=0.5, train_ts=500, val_ts=500, features_to_take=["n_flows", "n_packets"],
                           transform_with=fitted_transformer, partial_fit_initialized_transformer=True, nan_threshold=0.5, random_state=999)
series_based_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-26 20:07:48,614][series_config][INFO] - Quick validation succeeded.
[2025-08-26 20:07:48,645][series_config][INFO] - Finalization and validation completed successfully.
[2025-08-26 20:07:48,648][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time period.
100%|██████████| 1000/1000 [00:00<00:00, 1148.57it/s]
[2025-08-26 20:07:49,522][cesnet_dataset][INFO] - Dataset initialization complete. Configuration updated.
[2025-08-26 20:07:49,522][cesnet_dataset][INFO] - Config initialized successfully.



Config Details:
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_10_MINUTES
    Source: SourceType.IP_ADDRESSES_SAMPLE

    Time series
        Train time series IDS: [  4380  35210 322201    190 307400 ...  29194 617662 677144 211973 612051], Length=65
        Val time series IDS: [ 10197  97179 151737 121596 191587 ...  11069 360850 368641    103 182151], Length=70
        Test time series IDS None
        All time series IDS [  4380  35210 322201    190 307400 ...  11069 360850 368641    103 182151], Length=135
    Time periods
        Time period: range(0, 20149)
    Features
        Taken features: ['n_flows', 'n_packets']
        Default values: [0. 0.]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.ID_TIME
    Fillers         
        Filler type: None
    Transformers
        Transformer type: CustomTransformer (Custom)
        Are transformers premade: True
        Are premade transformers p

In [20]:
series_based_dataset.get_val_df(workers=0).head(10)

,id_ip,id_time,n_flows,n_packets
0,10197.0,0.0,0.000497,0.000035
1,10197.0,1.0,0.000530,0.000019
2,10197.0,2.0,0.000440,0.000033
3,10197.0,3.0,0.000542,0.000037
4,10197.0,4.0,0.000790,0.000240
5,10197.0,5.0,0.000372,0.000038
6,10197.0,6.0,0.000463,0.000033
7,10197.0,7.0,0.000643,0.000394
8,10197.0,8.0,0.000384,0.000047
9,10197.0,9.0,0.000418,0.000022
